In [17]:
from typing import Sequence, Literal
import random
import json
from langchain.agents import AgentType, initialize_agent
from langchain.llms import LlamaCpp,CTransformers
from langchain.tools import tool
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [5]:
with open("./data/appmap_tool_data.json","r") as f:
    appmap = json.load(f)    

In [10]:
ignore_attrs:Sequence=("session_ids",)
users_per_segment:int=2
label2users = appmap["label2users"]
@tool
def get_users(top_k:int=5,sort_by:Literal['mean_week_duration','total_keyflows']='total_keyflows',desc:bool=True):
    """
    Use this function to get list of users of application with different sorts.
    """
    if top_k == -1:
        top_k = len(appmap["users_info"])
    users_info = dict(
        sorted(
            appmap["users_info"].items(),
            key=lambda x: x[1][sort_by],
            reverse=desc,
        )[:top_k]
    )
    return {u_id:{attr:v for attr, v in attrs.items() if attr not in ignore_attrs} for u_id,attrs in users_info.items()}

@tool
def get_users_by_segments():  
    """
    Use this function to get user segment with couple example users for each segment.
    """  
    filtered = {int(key): random.sample(users,k=min(len(users),users_per_segment)) for key, users in label2users.items()}
    for k in filtered:
        users = filtered[k]
        filtered[k] = [{attr:v for attr, v in user.items() if attr not in ignore_attrs} for user in users]
    
    filtered = {f"Segment: {i}":v for i,v in enumerate(filtered.values())}    
    return filtered

In [38]:
# Callbacks support token-wise streaming
# Verbose is required to pass to the callback manager
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

# llm = LlamaCpp(
#     model_path="../models/eachadea_ggml-vicuna-13b-1.1/ggml-vic13b-q5_1.bin", 
#     n_ctx=512, 
#     callback_manager=callback_manager, 
#     verbose=True
# )

llm = CTransformers(
    # model="../models/falcon-7b-instruct-ggml/falcon7b-instruct.ggmlv3.q4_0.bin",
    # model_type="falcon", 
    # config={"temperature":0},
    model="../models/starchat-beta-ggml/starchat-beta.ggmlv3.q4_0.bin",
    model_type="starcoder", 
    callback_manager=callback_manager,
    verbose=True
    )
# TODO: Write correct prompt for starchat

In [40]:
tools = [
    get_users,
    # get_users_by_segments
]

agent = initialize_agent(
    tools, llm, agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [41]:

agent.run("Who are the power users?")



> Entering new  chain...


0
<|system|>
0
<|system|>


> Finished chain.


'0\n<|system|>\n'

In [39]:
prompt = """How do I sort a list in Python?"""
response = llm(prompt)


Can you give me an example of how to write a python function that sorts a list of numbers from smallest to largest, then prints it out to the screen.